In [ ]:
pip install "openai>=1.76.0" "pydantic>=2.10,<3" "griffe>=1.5.6,<2" "typing-extensions>=4.12.2,<5" "requests>=2.0,<3" "types-requests>=2.0,<3" "beautifulsoup4" "reportlab" "mcp>=1.6.0,<2; python_version >= '3.10'"

In [ ]:
pip install openai-agents

In [ ]:
from agents import Agent, ItemHelpers, MessageOutputItem, Runner, trace
import asyncio

english_agent = Agent(
    name="english_agent",
    instructions="You translate the user's message to English",
    handoff_description="A polish to english translator",
)

german_agent = Agent(
    name="german_agent",
    instructions="You translate the user's message to German",
    handoff_description="A polish to german translator",
)

italian_agent = Agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="A polish to italian translator",
)

In [ ]:
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools in order."
        "You never translate on your own, you always use the provided tools."
    ),
    tools=[
        english_agent.as_tool(
            tool_name="translate_to_english",
            tool_description="Translate the user's message to English",
        ),
        german_agent.as_tool(
            tool_name="translate_to_german",
            tool_description="Translate the user's message to German",
        ),
        italian_agent.as_tool(
            tool_name="translate_to_italian",
            tool_description="Translate the user's message to Italian",
        ),
    ],
)

In [ ]:
synthesizer_agent = Agent(
    name="synthesizer_agent",
    instructions="You inspect translations, correct them if needed, and produce a final concatenated response.",
)

In [63]:
msg = input("Co chciałbyś mieć przetłumaczone:")

with trace("Orchestrator evaluator"):
    orchestrator_result = await Runner.run(orchestrator_agent, msg)

    for item in orchestrator_result.new_items:
        if isinstance(item, MessageOutputItem):
            text = ItemHelpers.text_message_output(item)
            if text:
                print(f"  - Translation step: {text}")

    synthesizer_result = await Runner.run(
        synthesizer_agent, orchestrator_result.to_input_list()
    )

print(f"\n\nFinal response:\n{synthesizer_result.final_output}")

Co chciałbyś mieć przetłumaczone: Na niemiecki - dzisiaj jest piękny dzień na napalm


  - Translation step: "dzisiaj jest piękny dzień na napalm" translates to German as:

"Heute ist ein schöner Tag für Napalm."


Final response:
"Dzisiaj jest piękny dzień na napalm" translates to German as:

"Heute ist ein schöner Tag für Napalm."
